<a href="https://colab.research.google.com/github/christianhidber/easyagents/blob/master/jupyter_notebooks/easyagents_logging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Investigating an agent api through logging, seeding & fixing juypter output cell clearing

### Install packages (gym, tfagents, tensorflow,....)

#### suppress package warnings, in colab: load additional packages for rendering

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import warnings

warnings.filterwarnings('ignore')
if 'google.colab' in sys.modules:
    !apt-get install xvfb >/dev/null
    !pip install pyvirtualdisplay >/dev/null    
    
    from pyvirtualdisplay import Display
    Display(visible=0, size=(960, 720)).start() 
else:
    #  for local installation
    sys.path.append('..')

#### install easyagents and rendering for orso

In [ ]:
if 'google.colab' in sys.modules:
    !pip install -q easyagents >/dev/null
    !pip install -q networkx==2.3.0 >/dev/null

## Agent logging 

Use the log.Agent() callback to investigate how easyagents interacts with a backend:

In [1]:
from easyagents.agents import PpoAgent
from easyagents.callbacks import duration, log

ppoAgent = PpoAgent('CartPole-v0')
ppoAgent.train([log.Agent(), duration.Fast()], default_plots=False)

Plots clear the jupyter cell ouput before each update thereby clearing the log output as well, thus we turned them off.
Typically each call to the backend api during training is logged. 
Note that the logging starts with 'tfagents' the default backend for the PpoAgent.
We then see a sequence of calls performing the Agent initialisation before we enter the train loop.
Api calls during play or evaluation are not logged.

Let's take a look at the tensorforce backend:

In [1]:
from easyagents.agents import PpoAgent
from easyagents.callbacks import duration, log

ppoAgent = PpoAgent('CartPole-v0', backend='tensorforce')
ppoAgent.train([log.Agent(), duration.Fast()], default_plots=False)

While in tensorforce we also first do a sequence of agent and policy. Note that in contrast to tfagents we do not
build up actor and critic policy networks but instead pass a network specification to the Agent.create call.
Moreover tensorforce implements already the train loop through its Runner class. 
Thus we only see 1 call to runner.run instead of the many api calls for tfagents.

## Seeding

To set a seed use:

In [1]:
import easyagents

easyagents.agents.seed = 0

Once set, the seed is applied before each call to train. Let's validate this using our log.Agent callback:

In [2]:
from easyagents.agents import PpoAgent
from easyagents.callbacks import duration, log

ppoAgent = PpoAgent('CartPole-v0', backend='tensorforce')
ppoAgent.train([log.Agent(), duration.Fast()], default_plots=False)

Note that at the very beginning the calls to set the seeds for tensorflow, numpy and python.

## Gym steps logging
Use the log.Step() callback to investigate how the agent interacts with the gym environment:

In [3]:
from easyagents.agents import PpoAgent
from easyagents.callbacks import duration, log

ppoAgent = PpoAgent('CartPole-v0')
ppoAgent.train([log.Step(), duration.Fast()], default_plots=False)

For each call to the gym environments step method you get a log entry, along with the action taken and current
observation. Each entry starts with 

[{gym_env_id} {instance_id}:{episode_in_instance}:{step_in_episode}]

followed by the id of the current training iteration as well as the current iteration step count.
If in a evaluation period you get the same statistics for the current evaluation episode.

You may easily implement other log callbacks to produce statistics specific to your problem domain.

## Fixing a jupyter output cell clearing
It seems that jupyter / matplotlib backend changes its behaviour of outputing the current figure of an 
evaluated cell (if you can help here, please let use know by 
[creating an issue](https://github.com/christianhidber/easyagents/issues/new/choose)).

Nonetheless you may directly control easyagents jupyter ouput cell clearing behaviour through the plot.Clear()
callback:


In [ ]:
from easyagents.agents import PpoAgent
from easyagents.callbacks import duration, log

ppoAgent = PpoAgent('CartPole-v0')
ppoAgent.train([log.Clear(on_train=False,on_play=False), duration.Fast()])

If your plot gets "doubled" after cell evaluation set on_train / on_play to True, if it disappears to False.